# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
# Load packages
import pandas as pd

In [2]:
# Import exercise data set
json_df = pd.read_json('data/world_bank_projects.json')

In [ ]:
# # Explore data 
# print(json_df.head())
# print('------------------------------------------')
# print(json_df.shape)
# print('------------------------------------------')
# print(json_df.index)
# print('------------------------------------------')
# print(json_df.info())

In [3]:
# The 10 countries with most projects - Method 1
top_countries = json_df.countryshortname.value_counts()
top_10_countries = top_countries[:10]
print(top_10_countries)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Nepal                 12
Morocco               12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64


In [ ]:
# # The 10 countries with most projects - Method 2
# print(  json_df.groupby('countryshortname')['_id'].count().sort_values(ascending=False)[:10] )  

In [ ]:
# # Visualize projects counts for the top 10 countries 
# import matplotlib.pyplot as plt
# plt.subplot(1,2,1)
# top_10_countries.plot(kind='bar', title='Top 10 countries')

# # Visualize shape of project counts by country
# plt.subplot(1,2,2)
# top_countries.plot(kind='bar', title='All countries')
# plt.axis('off')
# plt.show()

In [4]:
# Find the top 10 major project themes (using column 'mjtheme_namecode')

# Extract column with major project themes
project_themes_all = json_df['mjtheme_namecode']

# Create series of project theme codes
theme_code_list = [project_theme['code'] for project_themes in project_themes_all for project_theme in project_themes]
theme_code_series = pd.Series(theme_code_list)

# Output top 10 themes
code_count = theme_code_series.value_counts()
code_count_series = pd.Series(code_count, name='Project Count')
code_count_series.index.name = 'Code'
print(code_count_series)

Code
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
3      15
Name: Project Count, dtype: int64


In [5]:
# Create a dictionary of theme codes and names
theme_name_list = [project_theme['name'] for project_themes in project_themes_all for project_theme in project_themes]
code_name_set = set(zip(theme_code_list, theme_name_list))
clean_dict = {theme[0]: theme[1] for theme in code_name_set if theme[1] != ''}
code_name_series = pd.Series(clean_dict, name='Name')
code_name_series.index.name = 'Code'
print(code_name_series)


Code
1                              Economic management
10                               Rural development
11    Environment and natural resources management
2                         Public sector governance
3                                      Rule of law
4         Financial and private sector development
5                            Trade and integration
6            Social protection and risk management
7                      Social dev/gender/inclusion
8                                Human development
9                                Urban development
Name: Name, dtype: object


In [6]:
# Create dataframe with Code, Name and project count
df_code_name_count = pd.concat([code_name_series, code_count_series], axis=1)
df_code_name_count = df_code_name_count.sort_values('Project Count', ascending = False).reset_index()
df_code_name_count.columns = ['Code', 'Name', 'Count']
print(df_code_name_count)

   Code                                          Name  Count
0    11  Environment and natural resources management    250
1    10                             Rural development    216
2     8                             Human development    210
3     2                      Public sector governance    199
4     6         Social protection and risk management    168
5     4      Financial and private sector development    146
6     7                   Social dev/gender/inclusion    130
7     5                         Trade and integration     77
8     9                             Urban development     50
9     1                           Economic management     38
10    3                                   Rule of law     15


In [8]:
# Clean json_dataframe 
# Create copy of the dataframe 
json_df_clean = json_df.copy()

# Create clean column for 'mjtheme_namecode'
project_themes_col_clean = []

for project_themes in json_df_clean['mjtheme_namecode']:
    project = []
    for theme in project_themes:
        if theme['name'] == '':
            project.append({'code': theme['code'], 'name': clean_dict[theme['code']]})
        else: 
            project.append(theme)
    project_themes_col_clean.append(project)

# Copy over clean column
json_df_clean['mjtheme_namecode'] = project_themes_col_clean

# Print to check 
print( 'Pre Clean:', str(json_df['mjtheme_namecode'][0]) )
print( 'Post Clean:', str(json_df_clean['mjtheme_namecode'][0]) )

Pre Clean: [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
Post Clean: [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': 'Environment and natural resources management'}]
